In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
data = pd.read_csv('/content/drive/MyDrive/healthcare-dataset-stroke-data.csv')
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [10]:
data.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,0.816895,0,1,0.801265,0.301260,1,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0
1,0.743652,0,0,0.679023,0.212981,1,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
2,0.975586,0,1,0.234512,0.254296,1,0,1,0,0,...,0,1,0,0,1,0,0,0,1,0
3,0.597168,0,0,0.536008,0.276060,1,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
4,0.963379,1,0,0.549349,0.156930,1,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [5]:
data = pd.read_csv('/content/drive/MyDrive/healthcare-dataset-stroke-data.csv')


In [6]:
# Handle missing values in 'bmi' column
data['bmi'].fillna(data['bmi'].mean(), inplace=True)

# One-hot encode categorical variables
data = pd.get_dummies(data, columns=['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'])

# Drop 'id' column as it's not useful for predictions
data.drop('id', axis=1, inplace=True)

# Standardize numerical variables using MinMaxScaler
scaler = MinMaxScaler()
data[['age', 'avg_glucose_level', 'bmi']] = scaler.fit_transform(data[['age', 'avg_glucose_level', 'bmi']])

# Separate input features and target variable
X = data.drop('stroke', axis=1)
y = data['stroke']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

model.fit(x=X_train, y=y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stop], verbose=1)


Epoch 1/100
128/128 [==============================] - 2s 4ms/step - loss: 0.2619 - accuracy: 0.9481 - val_loss: 0.2195 - val_accuracy: 0.9393
Epoch 2/100
128/128 [==============================] - 0s 3ms/step - loss: 0.1778 - accuracy: 0.9543 - val_loss: 0.2136 - val_accuracy: 0.9393
Epoch 3/100
128/128 [==============================] - 0s 3ms/step - loss: 0.1710 - accuracy: 0.9543 - val_loss: 0.1946 - val_accuracy: 0.9393
Epoch 4/100
128/128 [==============================] - 1s 5ms/step - loss: 0.1714 - accuracy: 0.9543 - val_loss: 0.1889 - val_accuracy: 0.9393
Epoch 5/100
128/128 [==============================] - 1s 4ms/step - loss: 0.1632 - accuracy: 0.9543 - val_loss: 0.1864 - val_accuracy: 0.9393
Epoch 6/100
128/128 [==============================] - 1s 4ms/step - loss: 0.1656 - accuracy: 0.9543 - val_loss: 0.1845 - val_accuracy: 0.9393
Epoch 7/100
128/128 [==============================] - 1s 4ms/step - loss: 0.1582 - accuracy: 0.9543 - val_loss: 0.1863 - val_accuracy: 0.9393

In [14]:
# Create dummy data
dummy_data = {
    'age': 95,
    'hypertension': 1,
    'heart_disease': 1,
    'avg_glucose_level': 100,
    'bmi': 35,
    'gender_Male': 1,
    'gender_Female': 0,
    'gender_Other': 0,
    'ever_married_No': 0,
    'ever_married_Yes': 1,
    'work_type_children': 0,
    'work_type_Govt_jov': 1,
    'work_type_Never_worked': 0,
    'work_type_Private': 0,
    'work_type_Self-employed': 0,
    'Residence_type_Rural': 0,
    'Residence_type_Urban': 1,
    'smoking_status_formerly smoked': 1,
    'smoking_status_never smoked': 0,
    'smoking_status_smokes': 0,
    'smoking_status_Unknown': 0
}

# Convert dummy data to pandas DataFrame and apply scaling
dummy_data_df = pd.DataFrame([dummy_data])
dummy_data_df[['age', 'avg_glucose_level', 'bmi']] = scaler.transform(dummy_data_df[['age', 'avg_glucose_level', 'bmi']])

# Make a prediction on the dummy data
dummy_prediction_prob = model.predict(dummy_data_df)

# Convert the probability into binary classes using a threshold of 0.5
threshold = 0.5
dummy_prediction_class = (dummy_prediction_prob > threshold).astype(int)

# Print the result
if dummy_prediction_class[0][0] == 1:
    print("The patient is predicted to have a stroke.")
else:
    print("The patient is predicted not to have a stroke.")


1/1 [==============================] - 0s 59ms/step
The patient is predicted not to have a stroke.


In [11]:
df = pd.read_csv('/content/drive/MyDrive/healthcare-dataset-stroke-data.csv')
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [12]:
df["stroke"].value_counts()

0    4861
1     249
Name: stroke, dtype: int64